In [1]:
import pandas as pd
import os,glob
import csv
import shutil
import json
import numpy as np
import math
import re
import itertools
from functools import reduce
from pathlib import Path
from collections import Counter
from datetime import datetime,timedelta
abs_file_path = os.path.abspath("./")
import pandas as pd
from datetime import datetime,timedelta
#from .glb_var import glb_var as gb
import itertools as it
from dateutil.relativedelta import *
#from .log_util import log
from fbprophet import Prophet
from datetime import datetime
import subprocess

Importing plotly failed. Interactive plots will not work.


In [2]:
from utils.input_data import InputDataFile
from utils.forecast import Forecast
from utils import log_util

C:\Users\abenav.sankar\Downloads\spotcast (1)(1)\spotcast\app-server\server\resources


In [3]:
##Read Input file
DataPath = "D:/SpotCast/Data/"
dataset = pd.read_csv(DataPath+"Sample_data.csv")

In [4]:
dataset.head()

,Customer Segment,Product Category,Region,dt,Sales
0,Consumer,Furniture,Central,1/1/2016,690.77
1,Consumer,Furniture,Central,2/1/2016,5303.17
2,Consumer,Furniture,Central,4/1/2016,1547.77
3,Consumer,Furniture,Central,6/1/2016,3234.12
4,Consumer,Furniture,Central,7/1/2016,205.98


In [5]:
def get_output_paths():
    try:
        basedir = str('D:')
        output_storage = "{basedir}/SpotCast/output_files/".format(basedir=basedir)
        datastore = "data_{file_name}.h5".format(file_name=os.listdir(DataPath)[0][0:len(os.listdir(DataPath)[0])-4])
        output_file_path = os.path.join("{}/{}".format(output_storage, datastore))

        return output_file_path, output_storage

    except Exception as e:
        log_util.log("Error in getting Output Paths:",e)

In [6]:
def json_to_csv(data,filename,output_folder):

    if not os.path.exists('{}/table_csv/'.format(output_folder)):
        os.makedirs('{}/table_csv/'.format(output_folder))
    data_file = open('{}/table_csv/{}.csv'.format(output_folder,filename), 'w')
    csv_writer = csv.writer(data_file)
    count = 0
    for each in data:
        if count == 0:
            header = each.keys()
            csv_writer.writerow(header)
            count += 1
        csv_writer.writerow(each.values())
    data_file.close()

In [7]:
def compare_count(old_data,new_data):

    temp = {}
    temp_new = {}
    count_list = list()
    for each in old_data['tabledata']:
        dim_key = "_".join([each['Dimensions'], each['Drivers']])
        temp[dim_key] = each['Count']
    for each in new_data['tabledata']:
        dim_key = "_".join([each['Dimensions'], each['Drivers']])
        temp_new[dim_key] = each['Count']
    for key in temp_new.keys():
        if key not in temp.keys():
            count = temp_new[key]
            count_list.append(count)
        else:
            if temp_new[key] == 0:
                count = 0
            elif temp[key] == 5:
                count = 0
            else:
                count = temp_new[key] + temp[key]
            temp_new[key] = count
            count_list.append(count)
    incremental = 0
    try:
        for i in new_data['tabledata']:
            i['Count'] = count_list[incremental]
            incremental = int(incremental + 1)
    except IndexError as e:
        log_util.log("-------Error in comparing flag count-------:{}".format(e), 'error')
    return new_data

In [8]:
### Not used ####
def dimension_metric(dataset_id):

    global DIMENSIONS
    dataset = Dataset.query.filter_by(id=dataset_id).first()
    catalog = dataset.metadata_list[0].catalog
    dim_hierarchy = dataset.metadata_list[0].hierarchy['dim_hierarchy']
    DIMENSIONS = [each['display_name'] for each in dim_hierarchy]
    metrics = [{'name': each['business_friendly_name'], "type": "numeric"} for each in catalog if
               each['datatype'] == 'numeric']
    date_col = [{'name': each['business_friendly_name'], "type": "datetime64"} for each in catalog if
                each['datatype'] == 'date']

    return DIMENSIONS,metrics,date_col,dataset.dataset_name,dataset.file_path

In [9]:
## object has to be in dictionary format
def sorted_nicely(l):
    """ Sorts the given iterable in the way that is expected.
    Required arguments:
    l -- The iterable to be sorted.
    """

    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

In [10]:
def delete_prev_output_files(path):
    folder = path
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [11]:
def get_change_in_metric(data):
    try:
        actual = data['actual']
        forecast = data['forecast']
        return actual - forecast
    except Exception as e :
        print(e)

def get_percentage_change_in_metric(data):
    try:
        actual = data['actual']
        forecast = data['forecast']
        return ((actual - forecast) / forecast) * 100
    except ZeroDivisionError:
        return 0
    except Exception as e:
        print(e)

def get_percentage_share_change(total_share, current_value):
    try:
        return (current_value / abs(total_share)) * 100
    except ZeroDivisionError :
        return 0
    except Exception as e:
        print(e)

def get_index_vs_fairshare(share_change, dimension_length):
    try:
        return share_change * dimension_length
    except Exception as e:
        print(e)

In [12]:
class DatasetPrepare():

    def prep_data(file_path):
        try:
            # print("Define start", datetime.now())

            #dataset_id = in_data['dataset_id']
            #dimensions, metrics, date_col, dataset_name,file_path = dimension_metric(dataset_id=dataset_id)
            dim_dic, uniq_dic = dict(), dict()
            sort_lst, dimensions, date, metrics = list(), list(), list(), list()
            
            dataframe = pd.read_csv(file_path)
            dataset_name = file_path.split("/")[len(file_path.split("/"))-1].split(".")[0]
            
            for each in dataframe.columns:
                if dataframe[each].dtypes == "object" or dataframe[each].dtypes == "datetime64":
                    try:
                        dataframe[each] = pd.to_datetime(dataframe[each])
                        val = {'name': each, 'type': 'datetime64', 'distinct': len(dataframe[each].unique())}
                        date.append(val)
                    except ValueError:
                        val = {'name': each, 'type': 'object', 'distinct': len(dataframe[each].unique()),
                               'unique_values': [x for x in sorted_nicely(l=list(dataframe[each].unique())) if str(x) != 'nan']}
                        dimensions.append(val)

                elif dataframe[each].dtypes == "int64" or dataframe[
                    each].dtypes == "float64" or dataframe[each].dtypes == "int32" or dataframe[
                    each].dtypes == "float32":
                    val = {'dimension_name': each, 'dimension_type': 'numeric','distinct': len(dataframe[each].unique())}
                    metrics.append(val)
                
                
                
            #dimension_unique_value = [{ i: sorted_nicely(l=list(dataframe[i].unique()))} for i in dimensions]
            #dim = {'dim_distinct':dimensions,"unique_values":dimension_unique_value}
            datatype = {'dimensions':dimensions, 'date': date, 'metrics': metrics,"dataset_name":dataset_name}
            return datatype

        except Exception as e:
            print(e)

In [13]:
Dim_Metric = DatasetPrepare.prep_data(file_path=DataPath+"Sample_data.csv")

In [14]:
def dimension_metric(dataset):

    #global DIMENSIONS
    dataset = Dataset.query.filter_by(id=dataset_id).first()
    catalog = dataset.metadata_list[0].catalog
    dim_hierarchy = dataset.metadata_list[0].hierarchy['dim_hierarchy']
    DIMENSIONS = [each['display_name'] for each in dim_hierarchy]
    metrics = [{'name': each['business_friendly_name'], "type": "numeric"} for each in catalog if
               each['datatype'] == 'numeric']
    date_col = [{'name': each['business_friendly_name'], "type": "datetime64"} for each in catalog if
                each['datatype'] == 'date']

    return DIMENSIONS,metrics,date_col,dataset.dataset_name,dataset.file_path

In [15]:
class Tree(dict):
    def __missing__(self, key):
        value = self[key] = type(self)()
        return value

In [16]:
Dim_Metric

{'dimensions': [{'name': 'Customer Segment',
   'type': 'object',
   'distinct': 4,
   'unique_values': ['Consumer',
    'Corporate',
    'Home Office',
    'Small Business']},
  {'name': 'Product Category',
   'type': 'object',
   'distinct': 3,
   'unique_values': ['Furniture', 'Office Supplies', 'Technology']},
  {'name': 'Region',
   'type': 'object',
   'distinct': 4,
   'unique_values': ['Central', 'East', 'South', 'West']}],
 'date': [{'name': 'dt', 'type': 'datetime64', 'distinct': 48}],
 'metrics': [{'dimension_name': 'Sales',
   'dimension_type': 'numeric',
   'distinct': 2113}],
 'dataset_name': 'Sample_data'}

In [17]:
dimensions = []
for i in Dim_Metric['dimensions']:
    dimensions.append(i['name'])

In [18]:
date = []
for i in Dim_Metric['date']:
    date.append(i['name'])

In [19]:
for i in Dim_Metric['date']:
    print(i['name'])

dt


In [20]:
metrics = []
for i in Dim_Metric['metrics']:
    metrics.append(i['dimension_name'])

In [21]:
class InputDataFile():

    def __init__(self, env="production"):
        '''
        Global variables are referenced here from the prod environment
        '''
        self.default_gb = gb[env]()
        self.output_folder = self.default_gb.output_folder
        self.Int_id_col = self.default_gb.Int_id_col
        self.Int_metric_col_prev = self.default_gb.Int_metric_col_prev

    def save_output_to_hdf(table_name,data):
        '''
        Save Output files to hdf files in local system if the file exists in hdf5 format in the datastore
        Input: Table, dataset(csv),dataset_id
        Output: output file in hdf5
        '''
        try:
            basedir = str('D:')
            output_storage = "{basedir}/SpotCast/output_files/".format(basedir=basedir)
            datastore = "data_{file_name}.h5".format(file_name=os.listdir(DataPath)[0][0:len(os.listdir(DataPath)[0])-4])
            #output_storage = "spotcast_datastore/output_files/user_{user_id}/{dataset_id}".format(user_id=user_id,dataset_id=dataset_id)
            #datastore = "user_{user_id}.h5".format(user_id=user_id)

            hdf_path = os.path.join("{}".format(output_storage))

            if os.path.isdir(hdf_path) is False:
                self.create_file_directory(hdf_path)

            if not self.check_if_file_exists(
                    "{hdf_path}/{hdf_store_name}".format(hdf_path=hdf_path, hdf_store_name=datastore)):

                hdf = pd.HDFStore(
                    "{hdf_path}/{hdf_store_name}".format(hdf_path=hdf_path, hdf_store_name=datastore))
                hdf.put(table_name, data, format='table',
                        data_columns=True, index=None)
                hdf.close()
            else:

                data.to_hdf("{hdf_path}/{hdf_store_name}".format(hdf_path=hdf_path, hdf_store_name=datastore),
                            table_name,index=None)
            return True
        except Exception as e:
            print("Exception while saving data-to-hdf")
            return False

    def create_file_directory(path):
        if not os.path.exists(path):
            os.makedirs(path)
            print("Directory ", path, " Created ")
        else:
            print("Directory ", path, " Already Exists")

    def check_if_file_exists(file_path):
        if os.path.exists(file_path):
            return True
        else:
            return False

    def input_data(data,met,date,dimension_list):
        '''
        Creates an input data stream with all possible combinations of groupby and sum operation applied on list of dimensions
        Input : dataset in csv, metric to sum, dimensions_list to group by, date to subset data
        Output : dataframe with possible combinations of grouped by dimension values
        '''
        #logging.info("Path : " + str(data))
        try:
            dimension = dimension_list
            dimension.append(date)
            result = pd.DataFrame()
            subset_list = [list(subset) for i in range(0, len(dimension) + 1) for subset in it.combinations(dimension, i)]
            x2 = [data.groupby(subset)[met].sum() for subset in subset_list  if len(subset) > 0] # grouping by subset of all possible combinations with sales
            x3 = [x.reset_index() for x in x2 ] # resetting index old index as columns and new index as index
            for x in x3:# concatinating empty df with x2
                result = pd.concat([result, x], axis=0, sort=False)
            result = result.dropna(subset=[date])
            #logging.info(result)
            result = result.dropna() #drop NA
            #log('----------Created Dataframe with all possible combinations----------', 'info')
            print('----------Created Dataframe with all possible combinations----------')
            return result
        except Exception as e:
            print(e)
            #log('----------Error in Input Data Function----------:{}'.format(e), 'error')

    def make_identifier(df):
        '''
        Extracts identifier column from the dataset as combination of first and second columns
        Input : Data in csv
        Output : unique identifier column values
        '''
        #logging.info("df : " + str(df))
        try:
            col_list = list(df.columns)
            string_id = df[col_list[0]].str.cat(df[col_list[1:]], sep="_")
            return pd.factorize(string_id)[0]  # --FACTORIZING WITH A NUMBER- FIND unique values
        except Exception as e:
            #log('----------Error in creating identifiers----------:{}'.format(e), 'error')
            print(e)
            
    def combination(results,met,date,dimension):
        '''
        Extracts identifier column from the dataset as combination of first and second columns
        Input : Data in csv
        Output : unique identifier column values
        '''

        try:
            results[Int_id_col] = InputDataFile.make_identifier(results[dimension]) + 1  # CREATING DIM_ID IN RESULT WITH ONLY STRING COLUMNS FROM FULL DATA
            # To create nodes for later usage
            nodes = results.drop([date,met], axis=1).drop_duplicates()

            # To get the nodes in the desired format
            InputDataFile.save_output_to_hdf(table_name="nodes",data=nodes)

            # To create connections for later usage
            result = results[[Int_id_col, date,met]]
            InputDataFile.save_output_to_hdf(table_name="cube",data=result)
            #log('----------Node and Cube dataframes saved to hdf file----------','info')
            print('----------Node and Cube dataframes saved to hdf file----------')
            return result
        except Exception as e:
            #log('----------Error in Combination Function----------:{}'.format(e), 'error')
            print(e)
            
    def data_preparation(met,date, period,cube):
        '''
        Extracts identifier column from the dataset as combination of first and second columns
        Input : Data in csv
        Output : unique identifier column values
        '''
        try:
            dataset = cube
            tp_data = dataset.pivot(index= date,columns=Int_id_col, values=met) #Reshapes data
            tp_data["date_index"] = pd.to_datetime(tp_data.index)
            # if resample_flag == 1:
            #     tp_data_ = tp_data.resample(period[0],on="date_index").sum()
            #     tp_data_["date_index"] = tp_data_.index.date
            # else:
            tp_data['date_index'] = tp_data['date_index'].dt.date
            tp_data_ = tp_data.sort_values(by="date_index")
            tp_data_ = tp_data_.fillna(0)
            actuals = tp_data_.iloc[len(tp_data_)-2:]
            actuals = actuals.transpose()
            actuals.columns = [Int_metric_col_prev, met]
            actuals = actuals[[met,Int_metric_col_prev]]
            InputDataFile.save_output_to_hdf(table_name="actuals",data=actuals)
            tp_data_=tp_data_.set_index(tp_data_["date_index"]) #set data index using date_index column

            if period[0] == 'MS':
                range = 12
            elif period[0] == 'D':
                range = 28
            elif period[0] == 'W':
                range = 7
            elif period[0] == 'Q':
                range = 4
            elif period[0] == 'Y':
                range = 4

            ini_time_for_now = tp_data_["date_index"].iloc[-1]
            dates = list()
            dates.append(ini_time_for_now)
            for i in dates:
                if period[0] == 'MS':
                    date_range = i + relativedelta(months=+1)
                elif period[0] == 'D':
                    date_range = i + relativedelta(days=+1)
                elif period[0] == 'W':
                    date_range = i + relativedelta(weeks=+1)
                elif period[0] == 'Q':
                    date_range = i + relativedelta(months=+3)
                elif period[0] == 'Y':
                    date_range = i + relativedelta(years=+1)
                dates.append(date_range)
                if len(dates) > range:
                    break
                else:
                    continue
            dates.pop(0)
            df = pd.DataFrame(dates)
            dateappend = tp_data_["date_index"].append(df, ignore_index=False)
            data = tp_data_.join(dateappend, how="outer")
            data=data.sort_values(by="date_index")
            data=data.drop(columns=['date_index'])
            tp_data_ = data.set_axis([*data.columns[:-1], 'Month'], axis=1, inplace=False)
            conv_dates = [date_obj.strftime('%Y-%m-%d') for date_obj in tp_data_['Month']]
            tp_data_['Month'] = conv_dates
            tp_data_=tp_data_.set_index(tp_data_['Month'])
            tp_data_1=tp_data_.drop(columns=['Month'])
            val=range+1
            transposed = tp_data_1.copy()
            tp_data_1.reset_index(inplace = True)
            InputDataFile.save_output_to_hdf(table_name="transposed_actuals",data=tp_data_1)
            transposed.iloc[len(tp_data_1) - val ] = np.NaN
            InputDataFile.save_output_to_hdf(table_name="transposed",data=transposed)
            #logging.info('Data Processed and written')
            #log('----------Transposed dataframe saved to hdf file----------','info')
            return val,ini_time_for_now,tp_data_1,transposed,actuals

        except Exception as e:
            #log('----------Error in Data preparation----------:{}'.format(e), 'error')
            print(e)
            
    def process(dimension_metric, metric, date, period):

        log('------Input Data Processing Started--------','info')
        #dimensions, metrics, date_col, datasetname, file_path = dimension_metric(dataset_id=dataset_id)
        data_ = input_data(data=pd.read_csv('{}'.format(file_path)), met=metrics, date=date,
                dimension_list=dimensions[:3])
        cube_dataset  = combination(results = data_,met=metric,date=date,dimension=dimensions[:3])
        value,actual_date,transposed_actuals ,transposed,actuals= data_preparation(met=metric,date=date, period=period,cube=cube_dataset,user_id=user_id,dataset_id=dataset_id)
        log('-------Input Data Processed and Done---------','info')
        return value,actual_date,transposed_actuals,transposed,actuals

In [22]:
def granularity_check(dataset, date_col,df_dt):
        try:
            date_tolist = list(set(dataset[date_col].values))
            date_tolist_2 = list(set(date_tolist))
            if df_dt is False:
                date_format ='%m/%d/%Y'
            elif df_dt is True:
                date_format = '%Y-%m-%d'
            date_tolist_3 = sorted(date_tolist_2, key=lambda x: datetime.strptime(str(x), date_format))
            dif_day = list()
            for i in range(5):
                dt1 = datetime.strptime(date_tolist_3[i + 1], date_format)
                dt2 = datetime.strptime(date_tolist_3[i], date_format)
                dif = dt1 - dt2
                dif_day.append(dif.days)
            len_dif = len(dif_day)
            prd = list()
            period = list()
            if len_dif == 1:
                prd = check_period(i)
                period.append(prd)
                return period
            elif len_dif > 1:
                for i in dif_day:
                    prd.append(check_period(i))
                res = max(set(prd), key=prd.count)
                period.append(res)
                # print("gran - 5", datetime.now())
        except Exception as e:
            #log_util.log("---------Error in getting latest date ------------:{}".format(e), 'error')
            raise e
        return period

In [23]:
def check_period(dif_day):
        if dif_day == 1:
            period = 'D'
            return period
        elif dif_day > 1 and dif_day <= 7:
            period = 'W'
            return period
        elif dif_day > 26 and dif_day <= 31:
            period = 'MS'
            return period
        elif dif_day > 85 and dif_day <= 95:
            period = 'Q'
            return period
        elif dif_day > 360 and dif_day <= 370:
            period = 'Y'
            return period

In [24]:
#dimension_list= dimensions[:3]
#subset_list = [list(subset) for i in range(0, len(dimension) + 1) for subset in it.combinations(dimension, i)]

In [25]:
df = dataset[dimensions]
col_list = list(df.columns)
string_id = df[col_list[0]].str.cat(df[col_list[1:]], sep="_")
pd.factorize(string_id)[0]

array([ 0,  0,  0, ..., 47, 47, 47], dtype=int64)

In [26]:
results = InputDataFile.make_identifier(dataset[dimensions])

In [27]:
dimensions

['Customer Segment', 'Product Category', 'Region']

In [28]:
dimension = dimensions
dimension.append(date[0])
result = pd.DataFrame()
subset_list = [list(subset) for i in range(0, len(dimension) + 1) for subset in it.combinations(dimension, i)]

In [29]:
x2[0].reset_index()

NameError: name 'x2' is not defined

In [30]:
subset_list

[[],
 ['Customer Segment'],
 ['Product Category'],
 ['Region'],
 ['dt'],
 ['Customer Segment', 'Product Category'],
 ['Customer Segment', 'Region'],
 ['Customer Segment', 'dt'],
 ['Product Category', 'Region'],
 ['Product Category', 'dt'],
 ['Region', 'dt'],
 ['Customer Segment', 'Product Category', 'Region'],
 ['Customer Segment', 'Product Category', 'dt'],
 ['Customer Segment', 'Region', 'dt'],
 ['Product Category', 'Region', 'dt'],
 ['Customer Segment', 'Product Category', 'Region', 'dt']]

In [31]:
date='dt'
result = pd.DataFrame()
x2 = [dataset.groupby(subset)[metrics].sum() for subset in subset_list  if len(subset) > 0]
x3 = [x.reset_index() for x in x2 ]
for x in x3:# concatinating empty df with x2
    result = pd.concat([result, x], axis=0, sort=False)
result = result.dropna(subset=[date])
#logging.info(result)
result = result.dropna()

In [32]:
result

,Customer Segment,Sales,Product Category,Region,dt
0,Consumer,690.77,Furniture,Central,1/1/2016
1,Consumer,1259.92,Furniture,Central,1/1/2017
2,Consumer,726.98,Furniture,Central,1/1/2018
3,Consumer,7368.24,Furniture,Central,1/1/2019
4,Consumer,1818.41,Furniture,Central,10/1/2016
5,Consumer,2254.48,Furniture,Central,10/1/2017
6,Consumer,440.01,Furniture,Central,10/1/2018
7,Consumer,1626.07,Furniture,Central,10/1/2019
8,Consumer,16.73,Furniture,Central,11/1/2017
9,Consumer,38312.01,Furniture,Central,11/1/2018


In [33]:
results= result
dimension= dimensions[:3]
date= 'dt'
met= 'Sales'
Int_id_col = 'dim_id'
result[Int_id_col] = InputDataFile.make_identifier(result[dimension]) + 1  # CREATING DIM_ID IN RESULT WITH ONLY STRING COLUMNS FROM FULL DATA
            # To create nodes for later usage
nodes = result.drop([date,met], axis=1).drop_duplicates()

            # To get the nodes in the desired format
#save_output_to_hdf(table_name="nodes",data=nodes)

            # To create connections for later usage
result = result[[Int_id_col, date,met]]
#save_output_to_hdf(table_name="cube",data=result)
            #log('----------Node and Cube dataframes saved to hdf file----------','info')
print('----------Node and Cube dataframes saved to hdf file----------')

----------Node and Cube dataframes saved to hdf file----------


In [34]:
result

,dim_id,dt,Sales
0,1,1/1/2016,690.77
1,1,1/1/2017,1259.92
2,1,1/1/2018,726.98
3,1,1/1/2019,7368.24
4,1,10/1/2016,1818.41
5,1,10/1/2017,2254.48
6,1,10/1/2018,440.01
7,1,10/1/2019,1626.07
8,1,11/1/2017,16.73
9,1,11/1/2018,38312.01


In [297]:
result

,dim_id,dt,Sales
0,1,1/1/2016,690.77
1,1,1/1/2017,1259.92
2,1,1/1/2018,726.98
3,1,1/1/2019,7368.24
4,1,10/1/2016,1818.41
5,1,10/1/2017,2254.48
6,1,10/1/2018,440.01
7,1,10/1/2019,1626.07
8,1,11/1/2017,16.73
9,1,11/1/2018,38312.01


In [35]:
data_ = InputDataFile.input_data(data=pd.read_csv('{}'.format(DataPath+"Sample_data.csv")), met=metrics, date=date,
                dimension_list=dimensions[:3])

----------Created Dataframe with all possible combinations----------


In [36]:
data_

,Customer Segment,Sales,Product Category,Region,dt
0,Consumer,690.77,Furniture,Central,1/1/2016
1,Consumer,1259.92,Furniture,Central,1/1/2017
2,Consumer,726.98,Furniture,Central,1/1/2018
3,Consumer,7368.24,Furniture,Central,1/1/2019
4,Consumer,1818.41,Furniture,Central,10/1/2016
5,Consumer,2254.48,Furniture,Central,10/1/2017
6,Consumer,440.01,Furniture,Central,10/1/2018
7,Consumer,1626.07,Furniture,Central,10/1/2019
8,Consumer,16.73,Furniture,Central,11/1/2017
9,Consumer,38312.01,Furniture,Central,11/1/2018


In [37]:
cube_dataset  = InputDataFile.combination(results = data_,met=metrics[0],date=date,dimension=dimensions[:3])

Exception while saving data-to-hdf
Exception while saving data-to-hdf
----------Node and Cube dataframes saved to hdf file----------


In [38]:
period = granularity_check(dataset,date,df_dt=False)

In [39]:
Int_metric_col_prev= "metric" + '_prev_week'
value,actual_date,transposed_actuals ,transposed,actuals= InputDataFile.data_preparation(met=metrics[0],date=date, period=period,cube=cube_dataset)

Exception while saving data-to-hdf
Exception while saving data-to-hdf
Exception while saving data-to-hdf


C:\Users\abenav.sankar\Anaconda3\envs\SpotCast\lib\site-packages\pandas\core\reshape\merge.py:765: RuntimeWarning: '<' not supported between instances of 'datetime.date' and 'int', sort order is undefined for incomparable objects
  sort=self.sort)


In [40]:
transposed_actuals

,Month,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,2016-01-01,690.77,5186.98,992.63,0.00,225.36,1112.11,1114.82,535.20,1345.52,...,2286.00,0.00,306.24,2327.89,1003.11,54.98,2411.60,5562.53,0.00,384.22
1,2016-02-01,5303.17,181.43,350.48,9819.63,62.80,84.01,420.18,233.51,4180.60,...,237.62,0.00,1923.90,956.19,32.74,218.72,381.96,2343.88,176.42,4814.83
2,2016-03-01,0.00,1828.22,2614.45,6776.86,419.37,0.00,1370.10,881.32,1497.22,...,5079.39,3722.41,1047.80,433.51,2402.78,476.76,3814.09,975.13,1489.61,899.06
3,2016-04-01,1547.77,11901.21,7413.55,0.00,252.73,395.12,923.90,387.46,1042.54,...,576.39,10194.05,293.69,862.75,14.66,1665.54,0.00,1444.39,367.52,1800.34
4,2016-05-01,0.00,4849.13,2860.05,9383.44,145.42,818.81,768.23,1732.11,0.00,...,0.00,2715.76,6075.51,13420.12,1795.69,699.82,479.79,3534.18,0.00,1886.97
5,2016-06-01,3234.12,3171.32,0.00,212.17,47.40,3044.96,7708.88,875.35,0.00,...,5012.80,1379.91,9322.78,13961.77,1032.36,79.93,1132.89,976.70,264.95,0.00
6,2016-07-01,205.98,318.18,1336.35,6499.87,269.10,51.64,2849.07,987.19,3855.28,...,2848.67,0.00,900.41,1452.90,5083.20,1168.66,1092.58,764.41,8332.91,7328.82
7,2016-08-01,8250.49,3596.03,894.88,7050.19,545.43,10655.13,0.00,399.08,1197.82,...,2815.54,6898.73,1094.63,80.58,460.24,620.79,0.00,2448.33,2875.35,777.73
8,2016-09-01,17.44,5362.72,198.67,668.38,1428.34,216.67,0.00,140.86,0.00,...,0.00,734.74,228.65,129.79,344.87,294.20,951.73,127.22,1348.83,4034.69
9,2016-10-01,1818.41,30706.84,2161.88,7705.12,1148.24,1777.79,2316.26,2139.97,296.76,...,26.66,0.00,15708.53,63327.17,922.35,0.00,0.00,5982.26,1331.22,2433.95


   **Data Preparation Ends here**

In [41]:
actuals

,Sales,metric_prev_week
dim_id,,
1,7323.44,5731.02
2,1281.55,1737.69
3,2983.03,4198.21
4,1554.58,3830.33
5,1391.94,385.83
6,8612.96,5615.07
7,314.37,1390.9
8,279.49,1155.72
9,3747.72,1725.8


In [305]:
def get_latest_date(period,dataset,metric,actuals):

        try:
            actual_out_dataset = actuals
            data_tail = actual_out_dataset.tail(1)
            actual_date = data_tail[metric].tolist()
            date_list = list(dataset['Month'])
            indexPos = [i for i, e in enumerate(date_list) if e == str(actual_date[0])][0]
            indexPos+=1
            actual_out_list = date_list[:indexPos]
            forecasting_out_list = date_list[(indexPos-1):]
            actual_dates_list = format_time(period,actual_out_list)
            forecasting_dates_list = format_time(period,forecasting_out_list)
            format_actual_date = actual_dates_list[-1]
            format_previous_date = actual_dates_list[-2]

        except Exception as e:
            #log_util.log("---------Error in getting latest date ------------:{}".format(e),'error')
            raise e
        return format_actual_date, format_previous_date, actual_dates_list, forecasting_dates_list

In [306]:
def format_time(format,data):

        timekey_list = list()
        if format =="MS":
            for each in data:
                timekey = datetime.strptime(each, "%Y-%m-%d")
                timekey = datetime.strftime(timekey, "%b-%Y")
                timekey_list.append(timekey)
        elif format =="W":
            for each in data:
                timekey = datetime.strptime(each, "%Y-%m-%d")
                timekey = datetime.strftime(timekey, "WK-%W (%b-%Y)")
                timekey_list.append(timekey)

        elif format =="D":
            for each in data:
                timekey = datetime.strptime(each, "%Y-%m-%d")
                timekey = datetime.strftime(timekey, "%Y-%m-%d (%a)")
                timekey_list.append(timekey)
        elif format =="Q":
            for each in data:
                timekey = datetime.strptime(each, "%Y-%m-%d")
                timekey = datetime.strftime(timekey, "Q%Q-%Y")
                timekey_list.append(timekey)
        elif format =="Y":
            for each in data:
                timekey = datetime.strptime(each, "%Y-%m-%d")
                timekey = datetime.strftime(timekey, "%Y")
                timekey_list.append(timekey)
        return timekey_list

In [307]:
latest_date, previous_date,actual_dates_list,forecasting_dates_list=get_latest_date(period[0],transposed_actuals,metrics[0],actuals)
actual_key = str("""{} {}""".format(latest_date, metric))
previous_key = str("""{} {}""".format(previous_date, metric))

In [311]:
class Forecasting:
    def __init__(self, dataset_path, dependent_metric, period , frequency , actual_date ,output_folder,transposed_actuals,transposed,user_id,dataset_id):
        '''
        :param hdf5_path: Path of the dataset
        :param table_name: Dataset name
        :param catalog: catalog contains information of the columns and values in dataset
        :param dependent_metric: The output variable metric
        :param choosen_dim: The dimension choosen from the user
        :param include_filter: Date filter and value filters
        '''
        self.dataset = dataset_path
        self.dependent_metric = dependent_metric
        self.period = period
        self.frequency = frequency
        self.actual_date = actual_date
        self.output_folder = output_folder
        self.date = ""
        self.transposed_actuals = transposed_actuals
        self.transposed = transposed
        self.input = input_data.InputDataFile()
        self.user_id = user_id
        self.dataset_id = dataset_id

    def resampling_data(self):
        '''
        Resampling the orginal data by groupby for the given metric

        '''
        # data = pd.read_csv(self.dataset, index_col=0)

        try:
            data = self.dataset
            new_data = data[self.dependent_metric].resample(self.frequency).sum()

        except Exception as e:
            log('----------Error in Resampling Data function ----------:{}'.format(e), 'error')
            raise e

        return new_data

    def clean_ts(self):
        """
        takes time series as input and returns cleaned time series

        *removes outliers
        *linearly interpolates missing values

        returns time series back
        """
        try:
            ts = self.resampling_data()
            # find the start of the time series
            start_ts = ts[ts.notna()].index[0]
            # find the end of the time series
            end_ts = ts[ts.notna()].index[-1]
            ts = ts.loc[start_ts:end_ts]

            # sets outliers to NaN defined as a point that is 3 deviations away from rolling mean + 3*rolling standard deviation
            ts = ts.where(~(abs(ts) > (ts.rolling(center=False, window=3).mean() + (
                        ts.rolling(center=False, window=3, min_periods=3).std() * 3))), np.nan)
            # linearly interpolate any NaN values
            ts = ts.interpolate()
            # make sure any remaining null values are filled with 0's
            ts = ts.fillna(0)


        except Exception as e:
            log('----------Error in Clean_ts function ----------:{}'.format(e), 'error')
            raise e

        return ts

    def forecast_prophet(self, ts):
        """forecasts a time series object using FB prophet model (https://facebook.github.io/prophet/)
        Note: forecast_ts returns fitted period values as well as forecasted period values

        if return_df is True need to get results like this:

            tseries, prophet_model, forc_df = forecast_prophet(df['series1'],return_df=True)

            Args:
                ts: time series object
                forecast_periods: periods to forecast for
                changepoint_prior_scale: changepoint value (lower value means trend is more flexible, easier to change)
                freq: frequency of the data ('MS' is Monthly Start could use M for month end)
                return_df: if set to true, returns full forecast dataframe with upper and lower values

            Returns: of f
                 forecast_ts: time seriesitted and forecasted values
                 forecast_df: full forecast dataframe to be used for plotting or for upper/lower conf interval values
                 model: prophet fitted model

        """
        try:
            forecast_periods = self.period
            changepoint_prior_scale = .05
            freq = self.frequency
            return_df = False
            # find the start of the time series
            start_ts = ts[ts.notna()].index[0]
            # find the end of the time series
            end_ts = ts[ts.notna()].index[-1]
            # extract actual time series
            ts = ts.loc[start_ts:end_ts]

            model_ts = ts.reset_index()
            model_ts.columns = ['ds', 'y']

            model = Prophet(changepoint_prior_scale=changepoint_prior_scale)
            model.fit(model_ts)

            future = model.make_future_dataframe(periods=forecast_periods, freq=freq)
            forecast_df = model.predict(future)

            forecast_ts = forecast_df.set_index('ds')['yhat']

            # date range is min of current series to max of all series (we'll filter out dataframe of series we dont want eventually so that we dont have to do this)

            if return_df == False:
                return forecast_ts
        except Exception as e:
            log('----------Error in Forecast_Prophet function ----------:{}'.format(e), 'error')
            raise e

        return forecast_ts

    def data_manupulation(self):
        '''
        Storing the predicted result in a csv format
        '''
        # output_fc = []
        # data = self.clean_ts()
        # forecasted_series = dask.delayed(self.forecast_prophet)(data)
        # output_fc.append(forecasted_series)

        # total = dask.delayed(output_fc).compute()
        try:
            data = self.clean_ts()

            data.to_csv("{}/actuals_out.csv".format(self.output_folder),header=['Actuals'])

            tseries = self.forecast_prophet(data)

            '''Converting the series data to dataframe with header predicted and storing in the r_outputs folder'''
            tseries.to_csv("{}/pred_out.csv".format(self.output_folder), header = ['Predicted'])
            predicted_result = pd.read_csv("{}/pred_out.csv".format(self.output_folder))

        except Exception as e:
            #log('----------Error in Data manipulation function ----------:{}'.format(e), 'error')
            raise e

        return predicted_result

    def data_ratio_data(self):
        '''
        getting the actual sum of the last row of the actual data to get the ratio
        '''
        try:
            predicted_result = self.data_manupulation()
            combination_data = self.transposed_actuals
            self.date = self.check_dates()
            last_row_actual = combination_data.loc[combination_data['Month'] == self.date]
            last_row_actual = last_row_actual.iloc[-1].tolist()
            total_sum_actual = sum(last_row_actual[1:])

        except Exception as e:
            #log('----------Error in data_ratio_data function ----------:{}'.format(e), 'error')
            raise e
        return total_sum_actual, last_row_actual[1:], predicted_result

    def find_ratios_actual(self):
        '''
        Getting the actual ratio from the actual data
        '''
        try:
            total_sum_actual, last_row_actual, predicted_result = self.data_ratio_data()
            output_ratio = list(map(lambda x: x / total_sum_actual, last_row_actual))
        except Exception as e:
            #log('----------Error in find_ratios_Actual function ----------:{}'.format(e), 'error')
            raise e

        return output_ratio, predicted_result

    def making_ratios(self):
        '''
        With the actual ratios we split the forecasted sum
        '''
        try:
            output_ratio, predicted_result = self.find_ratios_actual()

            df = predicted_result.loc[predicted_result['ds'] == datetime.datetime.strftime(self.actual_date,"%Y-%m-%d")]

            last_row_predicted = df.iloc[-1].tolist()

            sum_row_predicted = last_row_predicted[1]

            last_predicted_ratio = self.ratios_actual(output_ratio, sum_row_predicted)

        except Exception as e:
            #log('----------Error in making_ratios function ----------:{}'.format(e), 'error')
            raise e

        return last_predicted_ratio, output_ratio, predicted_result

    def replacing_ratios(self):
        '''
        Replacing the predicted ratio to the series dataset
        '''
        try:
            last_predicted_ratio, output_ratio, predicted_result = self.making_ratios()
            series = self.making_series_dataframe()
            series.loc[str(self.actual_date)] = last_predicted_ratio

        except Exception as e:
            #log('----------Error in replacing_ratios function ----------:{}'.format(e), 'error')
            raise e

        return series, output_ratio, predicted_result

    def making_all_ratios(self):
        '''
        Assigning the result to the nan values
        '''
        try:
            series, output_ratio, predicted_result = self.replacing_ratios()
            list_dates = self.getting_dates(series)

            for i in list_dates:
                sum_row_predicted = self.getting_predicted_value(predicted_result,i)
                series.loc[i] = self.ratios_actual(output_ratio, sum_row_predicted)
        except Exception as e:
            #log('----------Error in making_all_ratios function ----------:{}'.format(e), 'error')
            raise e

        return series

    def final_result(self):
        try:
            series = self.making_all_ratios()
            # series.to_hdf(self.output_folder + "transposed_otp.h5",key="transposed_otp",mode="w")
            self.input.save_output_to_hdf(user_id=self.user_id,table_name="transposed_otp",data=series,dataset_id=self.dataset_id)
        except Exception as e:
            #log('----------Error in Forecast Process ----------:{}'.format(e), 'error')
            raise e

        return series

    # def forecast_ratios(self):
    #     output_list, predicted_result = self.find_ratios_actual()
    #     last_row_actual = self.making_ratios()


    def ratios_actual(self, output, total_sum):
        try:
            output_ratio = list(map(lambda x: x * total_sum, output))
        except Exception as e:
            #log('----------Error in ratios_actual function ----------:{}'.format(e), 'error')
            raise e

        return output_ratio


    def making_series_dataframe(self):
        try:
            series = self.transposed
        except Exception as e:
            #log('----------Error in making_series_dataframe function ----------:{}'.format(e), 'error')
            raise e

        return series


    def getting_predicted_value(self, predicted_result, date):
        try:
            df = predicted_result.loc[predicted_result["ds"] == str(date)]
            last_row_predicted = df.iloc[-1].tolist()
            sum_row_predicted = last_row_predicted[1]

        except Exception as e:
            #log('----------Error in getting_predicted_value function ----------:{}'.format(e), 'error')
            raise e
        return sum_row_predicted

    def getting_dates(self, series):
        try:
            we = pd.isnull(series).any(1).nonzero()[0]
            output_ratio = list(map(lambda x: series.index[x], we))

        except Exception as e:
            #log('----------Error in getting_dates function ----------:{}'.format(e), 'error')
            raise e

        return output_ratio

    def check_dates(self):
        '''
        Checking for 'Month', 'Day' and 'Week' and assigning to the model for prediction and forecast
        '''
        try:
            today = datetime.datetime.strptime(str(self.actual_date), "%Y-%m-%d").date()
            if str(self.frequency) == 'D':
                last_monday = today - relativedelta(days=7)
            elif str(self.frequency) == 'MS':
                last_monday = today - relativedelta(months=1)
            elif str(self.frequency) == 'W':
                last_monday = today - relativedelta(weeks=1)
            elif str(self.frequency) == 'Q':
                last_monday = today - relativedelta(months=3)
            elif str(self.frequency) == 'Y':
                last_monday = today - relativedelta(years=1)
            date = last_monday.strftime("%Y-%m-%d")
        except Exception as e:
            #log('----------Error in check_Dates function ----------:{}'.format(e), 'error')
            raise e

        return date

In [312]:
class Forecast():# this approach need to be implemented in all process

    def __init__(self, env="production"):
        """
        Reads variables from the global environment 
        """
        self.default_gb = gb[env]()
        self.output_folder = self.default_gb.output_folder
        self.Int_id_col = self.default_gb.Int_id_col
        self.Int_metric_col_prev = self.default_gb.Int_metric_col_prev


    # def partitions(self,path,period):
    #
    #         logging.info("Path : " + str(path))
    #         retcode = subprocess.run(path)
    #         logging.info("Obtained a valid result R")
    #         # print(retcode.check_returncode())

    def insights(self,node,actuals,forecasted,val,met,actual_key,previous_key,actual_date,user_id,dataset_id):
        try:
            input = input_data.InputDataFile()
            transpose_ = pd.DataFrame(forecasted.transpose())
            transposed= transpose_[str(actual_date)]
            data = actuals[:-1]
            node[actual_key] = data[met]
            node[previous_key] = data[self.Int_metric_col_prev]
            node['Forecast'] = (transposed.iloc[0:]).values
            node[actual_key] = node[actual_key].astype(float)
            node[previous_key] = node[previous_key].astype(float)
            node['Forecast'] = node['Forecast'].astype(float)
            node.drop(columns=['dim_id'],inplace=True)
            node_index = node[(node[actual_key] == 0.0) & (node["Forecast"] == 0.0)].index
            node.drop(node_index,inplace=True)
            input.save_output_to_hdf(user_id=user_id,table_name="final_ds",data=node,dataset_id=dataset_id)
            log('----------Created Dataframe with Actual & Forecast values----------', 'info')

        except Exception as e:
            log('----------Error in Creating Final Dataframe----------: {}'.format(e), 'error')


    def lookup(self,s):
        """
        This is an extremely fast approach to datetime parsing.
        For large data, the same dates are often repeated. Rather than
        re-parse these, we store all unique dates, parse them, and
        use a lookup to convert all dates.
        """

        dates = {date: pd.to_datetime(date) for date in s.unique()}
        return s.map(dates)

    def process(self,dataset,metric,date,period,actual_key,previous_key,value,actual_date,transposed_actuals,transposed,user_id,output_folder_path,dataset_id):
        """
        Input: transposed dataset with combination of dimensions
        Output: Dataframe with actual and forecasted values 
        """
        try:
            dataset_path = dataset
            dataset_path[date] = self.lookup(dataset_path[date])
            dataset_path.set_index(date,inplace=True)

            log('----------Forecast Modelling - Started -----------', 'info')
            forecasting = Forecasting(dataset_path=dataset_path, dependent_metric=metric,
                                      period=value - 1,
                                      frequency=period[0], actual_date=actual_date, output_folder=output_folder_path,
                                      transposed_actuals=transposed_actuals, transposed=transposed,user_id=user_id,dataset_id=dataset_id)
            forecasting.final_result()
            log('----------Forecast Modelling - Done -----------', 'info')

            actuals = pd.read_hdf('{}/user_{user_id}.h5'.format(output_folder_path,user_id=user_id),"actuals")
            actuals.reset_index(inplace=True)
            nodes = pd.read_hdf('{}/user_{user_id}.h5'.format(output_folder_path,user_id=user_id),"nodes",index=False)
            nodes.reset_index(inplace=True)
            nodes = nodes.drop(columns=['index'])
            transposed_otp =  pd.read_hdf('{}/user_{user_id}.h5'.format(output_folder_path,user_id=user_id),"transposed_otp")
            self.insights(node=nodes,
                          actuals=actuals,
                          forecasted=transposed_otp,
                          val=value, met=metric, actual_key=actual_key, previous_key=previous_key,
                          actual_date=actual_date,user_id=user_id,dataset_id=dataset_id)

        except Exception as e:
            log('----------Error in Forecast Process ----------:{}'.format(e), 'error')

if __name__ == '__main__':

    forecast = Forecast()
    forecast.process()

NameError: name 'gb' is not defined